In [2]:
import pandas as pd 
import numpy as np 
import os 

In [3]:
path = "../../../../data/conv-trs/llm-results/Llama3Point2Vision90B_generated_parsed_queries.json"
df = pd.read_json(path)

In [4]:
df.head()

,config_id,config,context,city,query_v,query_p0,query_p1
0,c_p_0_pop_low_easy,"{'p_id': 'p_0', 'persona': 'A top-scoring play...",Adana has low popularity. Adana has low season...,"[Adana, Adiyaman, Agri, Arad, Arkhangelsk, Bac...","""Less crowded European cities to visit in Febr...","""European cities with ice hockey facilities, l...",Which European cities have ice hockey arenas a...
1,c_p_0_pop_low_medium,"{'p_id': 'p_0', 'persona': 'A top-scoring play...",Sivas has low popularity and low budget. In Si...,"[Sivas, Rivne, Konya, Craiova, Adana]","""Low budget European city breaks with old tree...","""European cities with low budget options and o...",Which European cities have ice hockey arenas a...
2,c_p_0_pop_low_hard,"{'p_id': 'p_0', 'persona': 'A top-scoring play...",Syktyvkar has low popularity and high budget. ...,"[Syktyvkar, Malatya, Kars, Ioannina]","""Unconventional European destinations with mus...","""European cities with low popularity, high bud...",Which European cities have ice hockey arenas a...
3,c_p_0_pop_low_sustainable,"{'p_id': 'p_0', 'persona': 'A top-scoring play...","Craiova has low popularity , low budget, and g...","[Craiova, Gaziantep]","""Low budget destinations in Europe with good a...","""Low budget European cities with great air qua...",Which European cities have ice hockey arenas a...
4,c_p_0_pop_medium_easy,"{'p_id': 'p_0', 'persona': 'A top-scoring play...",Aalborg has medium popularity. Aalborg has low...,"[Aalborg, Ancona, Astrakhan, Bari, Belgorod, B...","""Medium-sized European cities to visit in Janu...","""European cities with ice hockey facilities, m...",Which European cities have ice hockey arenas a...


In [5]:
def get_interest(config):
    return config if "interests" in config["filters"].keys() else None

configs = df['config'].apply(get_interest)

In [6]:
int_df = df[df['config'].isin(configs.tolist())]

In [7]:
def filter_easy(config_id):
    return None if "easy" in config_id else config_id 

non_easy_queries = int_df['config_id'].apply(filter_easy)
non_easy_df = int_df[int_df['config_id'].isin(non_easy_queries)]

In [8]:
non_easy_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1334 entries, 1 to 2301
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   config_id  1334 non-null   object
 1   config     1334 non-null   object
 2   context    1334 non-null   object
 3   city       1334 non-null   object
 4   query_v    1334 non-null   object
 5   query_p0   1334 non-null   object
 6   query_p1   1334 non-null   object
dtypes: object(7)
memory usage: 83.4+ KB


In [9]:
def retrieved_cities_10(city_list):
    return city_list if len(city_list) >= 10 else None 

final_queries = non_easy_df['city'].apply(retrieved_cities_10)
final_df = non_easy_df[non_easy_df['city'].isin(final_queries)]

In [10]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 626 entries, 9 to 2300
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   config_id  626 non-null    object
 1   config     626 non-null    object
 2   context    626 non-null    object
 3   city       626 non-null    object
 4   query_v    626 non-null    object
 5   query_p0   626 non-null    object
 6   query_p1   626 non-null    object
dtypes: object(7)
memory usage: 39.1+ KB


In [11]:
def find_level_pop(config_id):
    pop_level = config_id.split("_")[4:]
    return "_".join(pop_level)

final_df['pop_level'] = final_df['config_id'].apply(find_level_pop)

/var/folders/07/81pmlyln5szbq49h1zdqjq1h0000gn/T/ipykernel_42995/3246480299.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['pop_level'] = final_df['config_id'].apply(find_level_pop)


In [13]:
final_df['pop_level'].value_counts()

pop_level
[high, hard]             111
[high, medium]           101
[medium, medium]         100
[medium, hard]            97
[high, sustainable]       81
[medium, sustainable]     67
[low, medium]             55
[low, sustainable]        14
Name: count, dtype: int64

In [15]:
final_df['pop_level'] = final_df['pop_level'].astype(str)

/var/folders/07/81pmlyln5szbq49h1zdqjq1h0000gn/T/ipykernel_42995/554045634.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['pop_level'] = final_df['pop_level'].astype(str)


In [17]:
sampled = final_df.groupby('pop_level', group_keys=False).apply(lambda x: x.sample(5, random_state=42))

/var/folders/07/81pmlyln5szbq49h1zdqjq1h0000gn/T/ipykernel_42995/1330734960.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled = final_df.groupby('pop_level', group_keys=False).apply(lambda x: x.sample(5))


In [18]:
sampled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40 entries, 1556 to 630
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   config_id  40 non-null     object
 1   config     40 non-null     object
 2   context    40 non-null     object
 3   city       40 non-null     object
 4   query_v    40 non-null     object
 5   query_p0   40 non-null     object
 6   query_p1   40 non-null     object
 7   pop_level  40 non-null     object
dtypes: object(8)
memory usage: 2.8+ KB


In [29]:
sus_low_ids = sampled[sampled['pop_level'] == "['low', 'sustainable']"]['config_id'].tolist()

In [30]:
extra = final_df[(final_df['pop_level'] == "['low', 'sustainable']") & (~final_df['config_id'].isin(sus_low_ids))].sample(5, random_state=42)
extra.head()

,config_id,config,context,city,query_v,query_p0,query_p1,pop_level
512,c_p_44_pop_low_sustainable,"{'p_id': 'p_44', 'persona': 'A commercial loan...",Vladikavkaz has low popularity. Vladikavkaz ha...,"[Vladikavkaz, Vitoria-Gasteiz, Trabzon, Thessa...","""Off the beaten path destinations in Europe wi...","""European city breaks in December with low tou...",Which European cities offer a blend of financi...,"['low', 'sustainable']"
1456,c_p_126_pop_low_sustainable,"{'p_id': 'p_126', 'persona': 'A kindergarten t...",Van has low popularity. Van has low season in ...,"[Van, Uzhhorod, Trabzon, Thessaloniki, Sivas, ...","Looking for a low-budget, off-the-beaten-path ...","""European city breaks with low season in Janua...",Which European cities have interactive museums...,"['low', 'sustainable']"
626,c_p_54_pop_low_sustainable,"{'p_id': 'p_54', 'persona': 'A wanderlust-fill...",Vladikavkaz has low popularity. Vladikavkaz ha...,"[Vladikavkaz, Vitoria-Gasteiz, Uzhhorod, Trabz...",I'm ready to generate a query for the travel r...,"""European cities with low tourist season in No...",Which European cities are known for their loca...,"['low', 'sustainable']"
361,c_p_31_pop_low_sustainable,"{'p_id': 'p_31', 'persona': 'A novice cryptocu...",Vladikavkaz has low popularity. Vladikavkaz ha...,"[Vladikavkaz, Uzhhorod, Trabzon, Sivas, Samsun...","""Less crowded European cities to visit in Nove...","""Off the beaten path European destinations for...",Which European cities are easy to navigate for...,"['low', 'sustainable']"
604,c_p_52_pop_low_sustainable,"{'p_id': 'p_52', 'persona': 'Chinese historian...",Thessaloniki has low popularity and great walk...,"[Thessaloniki, Syktyvkar, Rivne, Malatya, Kony...",Looking for a less crowded European city with ...,"""European cities with low popularity and great...",Which European cities have well-preserved anci...,"['low', 'sustainable']"


In [31]:
sample = pd.concat([sampled, extra])

In [32]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45 entries, 1556 to 604
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   config_id  45 non-null     object
 1   config     45 non-null     object
 2   context    45 non-null     object
 3   city       45 non-null     object
 4   query_v    45 non-null     object
 5   query_p0   45 non-null     object
 6   query_p1   45 non-null     object
 7   pop_level  45 non-null     object
dtypes: object(8)
memory usage: 3.2+ KB


In [ ]:
8 different combinations - 1 each for sample run (no low-hard so it is replaced with low-sustainable instead)

In [25]:
import pandas as pd
import ast

In [7]:
samples_df = pd.read_csv("./data/conv-trs/multi-agent/sample-data/llama3point2_sample.csv")

In [32]:
sample = "c_p_5_pop_high_hard"
query_id = sample
query = samples_df.loc[samples_df.config_id == query_id, "query_v"].values[0]
travel_filters = ast.literal_eval(samples_df.loc[samples_df.config_id == query_id, "config"].values[0])["filters"]
city = samples_df.loc[samples_df.config_id == query_id, "city"].values[0]

In [30]:
offers = [
                "Paris",
                "Vienna",
                "Rome",
                "Barcelona",
                "Munich",
                "Geneva",
                "Amsterdam",
                "London",
                "Zurich",
                "Innsbruck"
            ]

In [34]:
[o for o in offers if o in city]

['Paris', 'Barcelona', 'Munich', 'Geneva', 'Amsterdam', 'London', 'Zurich']